## Initialization

In [ ]:
pip install requests pandas numpy scipy matplotlib plotly seaborn

In [ ]:
# libraries
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from scipy.optimize import fsolve
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from itertools import combinations
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# constants
AU = 1.496e11  # 1 AU in meters
mu = 1.32712440018e20  # Gravitational parameter for Sun (m^3/s^2)
earth_a = 1.0 * AU  # Earth's semi-major axis (1 AU)
earth_e = 0.0167  # Earth's eccentricity (small, almost circular orbit)
earth_epoch = 2459000.5  # Arbitrary epoch for Earth
earth_omega = 102.937  # Argument of perihelion for Earth (degrees)

In [ ]:
# Load the CSV file with object designations
file_path = 'cneos_sentry_summary_data.csv'   #dataset taken from NASA CNEOS website
data = pd.read_csv(file_path)
data.columns = data.columns.str.strip()
data['Object Designation'] = data['Object Designation'].str.replace(r'^\(|\)$', '', regex=True)

# Function to modify the object designations
def clean_object_designation(designation):
    # Remove extra spaces
    designation = ' '.join(designation.split())
    # Replace space after year with "%"
    parts = designation.split(" ")
    
    # Check if the first part is a year and if so, replace the following space
    if len(parts) > 1 and parts[0].isdigit() and len(parts[0]) == 4:  # Year check
        designation = parts[0] + "%20" + " ".join(parts[1:])  # Rejoin with `%`
    return designation

# Apply the function to the Object Designation column
data['Object Designation'] = data['Object Designation'].apply(clean_object_designation)

## Calculate object details


In [ ]:
def fetch_orbital_elements(object_designation):
    url = f"https://ssd-api.jpl.nasa.gov/sbdb.api?sstr={object_designation}&full-prec=true"
    response = requests.get(url)
        
    # Check for successful response
    if response.status_code == 200:
        data = response.json()

        # Check if 'orbit' exists 
        if 'orbit' in data and isinstance(data['orbit']['elements'], list):
            elements = data['orbit']['elements']
            print(f"Elements for {object_designation}: {elements}")  # Debugging line
            
            orbital_data = {elem['label']: elem['value'] for elem in elements if 'label' in elem and 'value' in elem}
            
            # Print the extracted orbital data for debugging
            print(f"Orbital data extracted for {object_designation}: {orbital_data}")  
            
            # Extracting orbital data 
            orbital_data = {elem['label']: elem['value'] for elem in elements if 'label' in elem and 'value' in elem}

            # return the correct keys
            return {
                'a': orbital_data.get('a'),        # Semi-major axis
                'e': orbital_data.get('e'),        # Eccentricity
                'i': orbital_data.get('i'),        # Inclination
                'Omega': orbital_data.get('node'),  # Longitude of ascending node
                'omega': orbital_data.get('peri'),   # Argument of perihelion
                'M0': orbital_data.get('M'),       # Mean anomaly
                'epoch': data['orbit'].get('epoch')  # Epoch from the orbit data
            }

    else:
        print(f"Error fetching data for {object_designation}: Status code {response.status_code}")

    return None

In [ ]:
# Add new columns for orbital elements in the DataFrame
data['a'] = pd.NA
data['e'] = pd.NA
data['i'] = pd.NA
data['Omega'] = pd.NA
data['omega'] = pd.NA
data['M0'] = pd.NA
data['epoch'] = pd.NA

# Loop over each object and fetch its orbital elements
# takes atmost 15 mins
for index, row in data.iterrows():
    object_designation = row['Object Designation']
    orbital_data = fetch_orbital_elements(object_designation)

    if orbital_data:
        data.at[index, 'a'] = float(orbital_data['a'])
        data.at[index, 'e'] = float(orbital_data['e'])
        data.at[index, 'i'] = float(orbital_data['i'])
        data.at[index, 'Omega'] = float(orbital_data['Omega'])
        data.at[index, 'omega'] = float(orbital_data['omega'])
        data.at[index, 'M0'] = float(orbital_data['M0'])
        data.at[index, 'epoch'] = float(orbital_data['epoch'])
    else:
        print(f"No orbital data found for {object_designation}")

data = data.dropna(subset=['a', 'e', 'i', 'Omega', 'omega', 'M0', 'epoch'])

In [ ]:
# Function to compute position based on orbital elements
def compute_position(a, e, i, Omega, omega, M0, epoch, t):
    # Constants and conversions 
    i = np.radians(i)
    Omega = np.radians(Omega)
    omega = np.radians(omega)
    M0 = np.radians(M0)

    # Mean motion
    n = np.sqrt(mu / a**3)
    dt = (t - epoch) * 86400  # Time difference in seconds
    M = M0 + n * dt  # Mean anomaly

    # Initial guess for eccentric anomaly
    E0 = M if e < 1 else np.pi  # For hyperbolic orbits, start at pi
    E = E0

    for _ in range(10):  # Maximum iterations
        E = E - (E - e * np.sin(E) - M) / (1 - e * np.cos(E))
        # Break if the change is small enough
        if abs(E - (E0 if e < 1 else np.pi)) < 1e-10:
            break

    # True anomaly
    true_anomaly = 2 * np.arctan2(np.sqrt(1 + e) * np.sin(E / 2), np.sqrt(1 - e) * np.cos(E / 2))

    # Distance to Sun
    r = a * (1 - e * np.cos(E))

    # 3D coordinates in the orbital plane
    x_prime = r * np.cos(true_anomaly)
    y_prime = r * np.sin(true_anomaly)

    # Convert to 3D space using the orbital elements
    cos_Omega = np.cos(Omega)
    sin_Omega = np.sin(Omega)
    cos_i = np.cos(i)
    sin_i = np.sin(i)
    cos_omega = np.cos(omega)
    sin_omega = np.sin(omega)

    x = (cos_Omega * cos_omega - sin_Omega * sin_omega * cos_i) * x_prime + (-cos_Omega * sin_omega - sin_Omega * cos_omega * cos_i) * y_prime
    y = (sin_Omega * cos_omega + cos_Omega * sin_omega * cos_i) * x_prime + (-sin_Omega * sin_omega + cos_Omega * cos_omega * cos_i) * y_prime
    z = (sin_i * sin_omega) * x_prime + (sin_i * cos_omega) * y_prime

    return np.array([x, y, z])


In [ ]:
#compute Earth's position
def compute_earth_position(t):
    # Mean anomaly for Earth (radians)
    M = (2 * np.pi / 365.25) * (t - 2451545)  # M = 2π/T * (t - epoch) 
    E0 = M  # Initial guess for E
    # Kepler's equation to find E
    def kepler(E, M):
        return E - np.sin(E) - M
    E = fsolve(kepler, E0, args=(M,))[0]  

    # Calculate true anomaly
    true_anomaly = 2 * np.arctan2(np.sqrt(1 + 0.0167) * np.sin(E / 2), np.sqrt(1 - 0.0167) * np.cos(E / 2))

    # Distance to the Sun
    r = AU  # Average distance from Earth to Sun (1 AU)

    # 3D coordinates in the orbital plane
    x = r * np.cos(true_anomaly)
    y = r * np.sin(true_anomaly)
    z = 0  

    return np.array([x, y, z])

t = 2459005.5  
earth_position = compute_earth_position(t)

## Calculate relative positions

In [ ]:
# Loop through the updated DataFrame and compute relative position to Earth
relative_positions = []  # To store relative positions

for index, row in data.iterrows():
    try:
        a = row['a'] * AU  # AU to meters
        e = row['e']
        i = row['i']
        Omega = row['Omega']
        omega = row['omega']
        M0 = row['M0']
        epoch = row['epoch']
        
        # Compute asteroid position
        asteroid_position = compute_position(a, e, i, Omega, omega, M0, epoch, t)
        
        # Calculate relative position to Earth
        relative_position = asteroid_position - earth_position
        relative_positions.append(relative_position)

    except Exception as err:
        print(f"Error at index {index}: {err}")

relative_positions = np.array(relative_positions)

# Add the relative positions to the DataFrame
data['relative_position_x'] = relative_positions[:, 0]  # x-coordinates
data['relative_position_y'] = relative_positions[:, 1]  # y-coordinates
data['relative_position_z'] = relative_positions[:, 2]  # z-coordinates

## Verify Data

In [ ]:
data.shape

In [ ]:
data.dtypes

In [ ]:
data.count()

## Data Re-Cleaning

In [ ]:
print(data.isnull().sum())

In [ ]:
data.describe()

In [ ]:
data.columns[data.isnull().any()]

In [ ]:
data.columns = data.columns.str.strip()
data.columns


In [ ]:
data.drop(['Unnamed: 10'],axis=1,inplace=True)
data[['Year Start', 'Year End']] = data['Year Range'].str.split("-", n=1, expand=True)

In [ ]:
data = data.dropna()    # Dropping the missing values.
data.count()


In [ ]:
data.head(2)

In [ ]:
data.drop(['Year Range'], axis=1, inplace=True)
numeric_columns = list(data.select_dtypes('number'))
numeric_columns

In [ ]:
data.head(2)

In [ ]:
data.shape

In [ ]:
data.dtypes

## Data Visualization

In [ ]:
# takes about a minute
plt.figure(figsize=(25, 25))
sns.pairplot(data=data)
sns.color_palette("mako", as_cmap=True)
plt.suptitle("Paired Histogram")
plt.show()

In [ ]:
data['Year Start'] = data['Year Start'].astype(int)
data['Year End'] = data['Year End'].astype(int)

In [ ]:
data['Time Interval (years)'] = data['Year End'] - data['Year Start']
data['Relative Position (km)'] = data['Vinfinity (km/s)'] * data['Time Interval (years)'] * 365 * 24 * 3600

data

In [ ]:
# Create a 3D scatter plot
fig = px.scatter_3d(data, 
                     x='relative_position_x', 
                     y='relative_position_y', 
                     z='relative_position_z', 
                     color='Object Designation',  # Use your appropriate column name for designation
                     title='Relative Positions of Asteroids',
                     labels={'relative_position_x': 'X Position (m)',
                             'relative_position_y': 'Y Position (m)',
                             'relative_position_z': 'Z Position (m)'},
                     hover_name='Object Designation',  # This will show designation on hover
                     size_max=10)  # Adjust size_max as necessary

# Show the plot
fig.show()


## Calculating Close Orbital Approach

In [ ]:
# ensure correct dtype
data['i'] = pd.to_numeric(data['i'], errors='coerce')

In [ ]:
def predict_position(x, y, z, vx, vy, vz, time):
    return x + vx * time, y + vy * time, z + vz * time

data['vx'] = data['Vinfinity (km/s)'] * np.cos(np.radians(data['i']))
data['vy'] = data['Vinfinity (km/s)'] * np.sin(np.radians(data['i']))
data['vz'] = 0  # Simplified assumption; adjust as necessary

def close_approach_analysis(data, threshold_distance=451234123953, time_interval=5):
    close_approaches = []
    min_year = data['Year Start'].min()
    max_year = data['Year End'].max()

    # Check only a subset of time steps
    time_steps = range(min_year, max_year, time_interval)

    # Iterate over all pairs of objects
    for i, j in combinations(data.index, 2):
        for year in time_steps:
            t_i = year - data.loc[i, 'Year Start']
            t_j = year - data.loc[j, 'Year Start']

            # Predict positions
            x1, y1, z1 = predict_position(data.loc[i, 'relative_position_x'], data.loc[i, 'relative_position_y'], data.loc[i, 'relative_position_z'],
                                          data.loc[i, 'vx'], data.loc[i, 'vy'], data.loc[i, 'vz'],
                                          t_i)
            x2, y2, z2 = predict_position(data.loc[j, 'relative_position_x'], data.loc[j, 'relative_position_y'], data.loc[j, 'relative_position_z'],
                                          data.loc[j, 'vx'], data.loc[j, 'vy'], data.loc[j, 'vz'],
                                          t_j)

            # Calculate distance
            dist = np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2 + (z1 - z2) ** 2)

            # Check if distance is below the threshold
            if dist < threshold_distance:
                close_approaches.append((data.loc[i, 'Object Designation'], data.loc[j, 'Object Designation'], year, dist))
    
    #         checking for finding threshold value (if needed)
    #         print(f'Checking pair: {data.loc[i, "Object Designation"]} and {data.loc[j, "Object Designation"]}')
    #         print(f'Positions: {x1}, {y1}, {z1} and {x2}, {y2}, {z2}')
    #         print(f'Distance: {dist}')

    return close_approaches


In [ ]:
# takes atmost 8 mins
close_approaches = close_approach_analysis(data)

In [ ]:
close_approach_data = pd.DataFrame(close_approaches, columns=['Object 1', 'Object 2', 'Year (approx.)', 'Distance (approx.)'])
print(close_approach_data)

In [ ]:
data['Risk Label'] = (data['Impact Probability (cumulative)'] > 4.00000e-07).astype(int)  # spproximate threshold
data = pd.get_dummies(data, drop_first=True)  # Convert categorical variables to dummy variables
X = data[['Vinfinity (km/s)', 'Estimated Diameter (km)', 'Impact Probability (cumulative)']]  # Features
y = data['Risk Label']  # Labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

y_true=y_test


In [ ]:
print(classification_report(y_test, y_pred))


In [ ]:
new_data = pd.DataFrame({
    'Vinfinity (km/s)': [5.0],
    'Estimated Diameter (km)': [0.051],
    'Impact Probability (cumulative)': [3.000000e-07]
})
new_data.fillna(data.mean(), inplace=True)  # Fill missing values
prediction = model.predict(new_data)
print("Risk Level:", prediction)  # 0 = low risk, 1 = high risk


In [ ]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=2)
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)


In [ ]:
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Incorrect', 'Correct'], 
            yticklabels=['Incorrect', 'Correct'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


In [ ]:
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10, 6))
plt.title('Feature Importances')
plt.bar(range(X.shape[1]), importances[indices], align='center')
plt.xticks(range(X.shape[1]), np.array(['Vinfinity (km/s)', 'Estimated Diameter (km)', 'Impact Probability (cumulative)'])[indices], rotation=90)
plt.xlim([-1, X.shape[1]])
plt.show()



In [ ]:
# Convert 'Year Start' to datetime 
data['Year Start'] = pd.to_datetime(data['Year Start'], format='%Y')

# Count close approaches by year
trend_data = data.groupby(data['Year Start'].dt.year)['Potential Impacts'].sum()

# Plot the trend
plt.figure(figsize=(10, 6))
trend_data.plot(kind='line', marker='o')
plt.title('Trend of Close Approaches Over Time')
plt.xlabel('Year')
plt.ylabel('Number of Close Approaches')
plt.grid()
plt.show()


## Clustering based on orbital elements (K-mmeans)

In [ ]:
# Select relevant features for clustering
features = data[['Vinfinity (km/s)','Estimated Diameter (km)']]  # Adjust columns as necessary

# Standardize the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

inertia = []
k_values = range(1, 11)

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(scaled_features)
    inertia.append(kmeans.inertia_)

# Calculate the differences in inertia
inertia_diff = np.diff(inertia)

# Find the index of the maximum change
elbow_index = np.argmax(np.diff(inertia_diff)) + 1  

# Get the corresponding number of clusters
optimal_k = k_values[elbow_index]

plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), inertia, marker='o')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.grid()
plt.show()

print(f'Optimal number of clusters (elbow): {optimal_k}')


In [ ]:
# Fit K-Means
kmeans = KMeans(n_clusters=2, random_state=42)  
data['Cluster'] = kmeans.fit_predict(scaled_features)

# Plotting the clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(data=data, x='Estimated Diameter (km)', y='Vinfinity (km/s)', hue='Cluster', palette='Set1')
plt.title('Clustering of Asteroids based on Diameter and Velocity')
plt.xlabel('Estimated Diameter (km)')
plt.ylabel('Vinfinity (km/s)')
plt.legend(title='Cluster')
plt.grid()
plt.show()